In [2]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup as bs

### Flight

In [3]:
data = requests.get("""https://api.skypicker.com/flights?\
fly_from=city:BRS&\
fly_to=city:PRG&\
date_from=19/12/2019&\
date_to=23/12/2019""")

In [4]:
Df = json.loads(data.content)

In [5]:
Df = json_normalize(Df["data"])

In [6]:
Df = Df[Df.apply(lambda x: x["route"][0]['cityTo'] == "Prague", axis=1)].sort_values(by="price")
Df

,aTime,aTimeUTC,airlines,baglimit.hand_height,baglimit.hand_length,baglimit.hand_weight,baglimit.hand_width,baglimit.hold_dimensions_sum,baglimit.hold_height,baglimit.hold_length,...,p1,p2,p3,pnr_count,price,quality,route,routes,transfers,virtual_interlining
6,1577118000,1577114400,[U2],45,56,15,25,275.0,90,135,...,1,1,1,1,96,123.99993,"[{'id': '1f2d145c474d00005f955bd3_0', 'combina...","[[BRS, PRG]]",[],False
10,1576772400,1576768800,[U2],45,56,15,25,275.0,90,135,...,1,1,1,1,101,128.99993,"[{'id': '1f2d145c47490000f5842621_0', 'combina...","[[BRS, PRG]]",[],False
19,1576858800,1576855200,[U2],45,56,15,25,275.0,90,135,...,1,1,1,1,110,137.99993,"[{'id': '1f2d145c474a0000765c8c9b_0', 'combina...","[[BRS, PRG]]",[],False
23,1577045700,1577042100,[U2],45,56,15,25,275.0,90,135,...,1,1,1,1,115,142.99993,"[{'id': '1f2d145c474c0000a81d7b11_0', 'combina...","[[BRS, PRG]]",[],False
30,1576922700,1576919100,[U2],45,56,15,25,275.0,90,135,...,1,1,1,1,123,150.99993,"[{'id': '1f2d145c474b000092dbbb1f_0', 'combina...","[[BRS, PRG]]",[],False


In [7]:
booking_token = Df.iloc[0, 14]

In [8]:
data = requests.get(f"""https://booking-api.skypicker.com/api/v0.1/check_flights?\
v=2&bnum=1&pnum=1&affily=some_id&\
booking_token={booking_token}""")

In [9]:
json.loads(data.content)

{'server_time': 1560237279,
 'pnum': 1,
 'flights': [{'id': '1f2d145c474d00005f955bd3_0',
   'combination_trip_id': '1f2d145c474d00005f955bd3',
   'original_trip_id': '1f2d145c474d00005f955bd3',
   'atime': 1577118000,
   'dtime': 1577107200,
   'atime_utc': 1577114400,
   'dtime_utc': 1577107200,
   'atime_unix': 1577118000,
   'dtime_unix': 1577107200,
   'atime_unix_utc': 1577114400,
   'dtime_unix_utc': 1577107200,
   'dst': 'PRG',
   'src': 'BRS',
   'flight_no': '6265',
   'airline': {'id': 3,
    'code': 'U2',
    'name': 'easyJet',
    'url': 'http://www.easyjet.com',
    'checkin': 30,
    'close_booking_hours': 4,
    'booking_doc_needed': 2,
    'airport_checkin': 12,
    'passengers_in_search': 1,
    'doing_online_checkin': 1,
    'maximum_passengers': 9,
    'grade': 'A',
    'virtual_card_req': False,
    'country': 'UK',
    'carrier_type': 'airline',
    'parent_carrier': None,
    'checkin_closure': 2.0,
    'shorter_stopovers_allowed': 1,
    'deprecated': False,
   

### Laptops

In [10]:
professional = requests.get("https://www.alza.co.uk/professional-laptops/18853299.htm")
home_office = requests.get("https://www.alza.co.uk/basic-home-and-office-laptops/18843464.htm")
gaming = requests.get("https://www.alza.co.uk/gaming-laptops/18848814.htm")

In [20]:
def npp(soup):
    def zerotext(l):
        try:
            data = []
            for i in range(len(l)):
                data.append(l[i].text)
            return data
        except IndexError:
            return None
    return zerotext(soup.select(".firstRow .name")), \
           zerotext(soup.select(".firstRow .np2")), \
           zerotext(soup.select(".firstRow .c2"))

In [21]:
pd.DataFrame(list(npp(bs(professional.text))))

,0,1,2
0,Lenovo V330-14IKB Iron Grey,Dell Latitude 5590,Lenovo ThinkPad T480
1,£618.90,"£1,502.90","£1,969"
2,£570.90,"£1,084.90","£1,929"


In [22]:
pd.DataFrame(list(npp(bs(home_office.text))))

,0,1,2
0,Microsoft Surface Go 128GB 8GB LTE,Dell Latitude 5590,Dell G7 17 (7790) Gaming Black
1,"£1,502.90","£1,939",None
2,£659,"£1,084.90","£1,629"


In [23]:
pd.DataFrame(list(npp(bs(gaming.text))))

,0,1,2
0,Lenovo IdeaPad 720s-15IKB Silver Metallic,Lenovo Legion Y530-15ICH Black,Dell G7 17 (7790) Gaming Black
1,£979,"£1,325.90","£1,939"
2,£889,"£1,220.90","£1,629"
